In [2]:
!pip install pytesseract
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!pip install deskew
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
from deskew import determine_skew
import os
from math import radians, cos, sin, asin, sqrt,pi
from pathlib import Path
from osgeo import gdal
import re
import json

  Created wheel for pytesseract: filename=pytesseract-0.3.4-py2.py3-none-any.whl size=13431 sha256=fe0a8067f9e24df720a1103fcb08c5181d5659266139d5e2da28b54021d26333
  Stored in directory: /root/.cache/pip/wheels/63/2a/a0/7596d2e0a73cf0aeffd6f6170862c4e73f3763b7827e48691a
Successfully built pytesseract
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,

In [3]:
!pip install gdown

In [5]:
# !gdown https://drive.google.com/uc?id=1--YsV2rgCLll6DH2k7ZGTutXizTVKC86
# !gdown https://drive.google.com/uc?id=1-0uqXyzqvgaEWAWQwFqeOpWPMLUFI8Jd
# !gdown https://drive.google.com/uc?id=1--BTKr_E1-eNqsF-m0ZgW4WnCMB09A6K
!gdown https://drive.google.com/uc?id=1zfIOVHByz1DTLNqd2DtQbpEEp3BSOayX
# !gdown https://drive.google.com/uc?id=1vmriQvb3OUvKCbUbiYqRory615i_Qm9z
# https://drive.google.com/drive/folders/1vmriQvb3OUvKCbUbiYqRory615i_Qm9z?usp=sharing
# https://drive.google.com/drive/folders/1vmriQvb3OUvKCbUbiYqRory615i_Qm9z?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1zfIOVHByz1DTLNqd2DtQbpEEp3BSOayX
To: /content/Scans.zip
17.7GB [04:35, 64.3MB/s]


In [0]:
# !tar -xvf '/content/map_tiles_pt1.tar.gz' -C '/'
# !tar -xvf '/content/map_tiles_pt2.tar.gz' -C '/'
# !tar -xvf '/content/xml.tar.gz' -C '/'

In [0]:
!unzip txt_predictions.zip #Rob's OCR base points
!unzip Scans.zip

In [16]:
cities = list(set(os.listdir('/content/')) - set(['results','sample_data','txt_predictions','Scans.zip','txt_predictions.zip','.config','.ipynb_checkpoints']))

def getCityExt(city):
  base = "/content/"+city
  b = (os.listdir(base))[0]
  base = base+"/"+b
  b = (os.listdir(base))
  lst = []
  for item in b:
    lst.append(base+"/"+item)
  return lst

def findAllFileIds(city):
  fileIds = []
  basePaths = getCityExt(city)
  for item in basePaths:
    prjFiles = list(filter(lambda x: x.endswith(".prj"),os.listdir(item)))
    prjFiles = list(map(lambda x: (item+"/",Path(x).stem),prjFiles))
    fileIds.extend(prjFiles)
  return fileIds

def setup(city):
  fileIds = findAllFileIds(city)
  for path,id_ in fileIds:
    gdal.Translate(path+id_ + '.png',
              path+id_ + '.tif',
              options="-of PNG")

/content/Middlesbrough/1056/tp_1/4030000000071
/content/Middlesbrough/1056/tp_1/4030000000021
/content/Middlesbrough/1056/tp_1/4030000000031
/content/Middlesbrough/1056/tp_1/4030000000081
/content/Middlesbrough/1056/tp_1/4030000000051
/content/Middlesbrough/1056/tp_1/4030000000061
/content/Middlesbrough/1056/tp_1/4030000000091
/content/Middlesbrough/1056/tp_1/4030000000041
/content/Middlesbrough/1056/tp_1/4030000000011
/content/Middlesbrough/1056/tp_1/4030000000101


In [0]:
!mkdir results

In [34]:
def allImagesToText():
  for city in cities:
    setup(city)
    maps = findAllFileIds(city)
    lat,lon = getBaseLatLon(city)
    for path,id_ in maps:
      appendLatLonInfo(city,path,id_,lat,lon)
    print(city + " DONE")

def appendLatLonInfo(city,path,id_,lat,lon):
  lines = None
  uId = city+"_"+id_
  try:
    with open("/content/txt_predictions/"+uId+".txt",'r') as f:
      lines = f.readlines()
  except Exception as e:
    return False

  latLons = list(map(lambda x: transformLineToLatLon(city,id_,path,x,lat,lon),lines))
  with open("/content/results/"+uId+".txt",'a') as f:
    for line in latLons:
      f.write(line)
  return True

def transformLineToLatLon(city,id_,path,l,lat,lon):
  pixelCoords = retrieveImageRequiredMetrics(l)
  pixelCoords = list(map(lambda x: mapImagePixelToLatLon(city,id_,path,x,lat,lon),pixelCoords))
  newX,newY = mapBboxToSingleCoord(pixelCoords)

  otherInfo = ","+",".join(l.split(",")[-4:])
  newLine = str(newX)+","+str(newY)+otherInfo
  return newLine

def retrieveImageRequiredMetrics(textline):
  pointsList = list(map(lambda x: int(float(x)), textline.split(",")[:-4]))
  xs = pointsList[0::2]
  ys = pointsList[1::2]
  coords = list(zip(xs,ys))
  return coords

allImagesToText()

Streaming output truncated to the last 5000 lines.
/content/Bristol/500/tp_1/1301075040151
/content/Bristol/500/tp_1/1301075030201
/content/Bristol/500/tp_1/1301075080061
/content/Bristol/500/tp_1/1301072090251
/content/Bristol/500/tp_1/1301072090131
/content/Bristol/500/tp_1/1301071160031
/content/Bristol/500/tp_1/1301076010151
/content/Bristol/500/tp_1/1301075040061
/content/Bristol/500/tp_1/1301071160221
/content/Bristol/500/tp_1/1301071120211
/content/Bristol/500/tp_1/1301071120181
/content/Bristol/500/tp_1/1301071120101
/content/Bristol/500/tp_1/1301071160141
/content/Bristol/500/tp_1/1301075080131
/content/Bristol/500/tp_1/1301075080041
/content/Bristol/500/tp_1/1301072090171
/content/Bristol/500/tp_1/1301075040011
/content/Bristol/500/tp_1/1301071150151
/content/Bristol/500/tp_1/1301075080071
/content/Bristol/500/tp_1/1301072130221
/content/Bristol/500/tp_1/1301071160081
/content/Bristol/500/tp_1/1301075040041
/content/Bristol/500/tp_1/1301071120061
/content/Bristol/500/tp_1/130

In [37]:
!zip -r results.zip results

  adding: results/ (stored 0%)
  adding: results/Leeds_4028218010151.txt (deflated 61%)
  adding: results/Birkenhead_0502007150241.txt (deflated 60%)
  adding: results/Leeds_4028218060241.txt (deflated 59%)
  adding: results/Nottingham_2705038100021.txt (deflated 60%)
  adding: results/Leicester_2003031140011.txt (deflated 60%)
  adding: results/Newport_5003028160241.txt (deflated 59%)
  adding: results/Northampton_2502045020211.txt (deflated 58%)
  adding: results/London_2201014000182.txt (deflated 61%)
  adding: results/London_2201007000182.txt (deflated 59%)
  adding: results/Birkenhead_0502013030071.txt (deflated 59%)
  adding: results/London_2201011000832.txt (deflated 59%)
  adding: results/York_4048174070211.txt (deflated 61%)
  adding: results/Nottingham_2705037160051.txt (deflated 57%)
  adding: results/Bristol_1301075040131.txt (deflated 60%)
  adding: results/Sheffield_4042295050081.txt (deflated 59%)
  adding: results/Blackburn_1906070030051.txt (deflated 60%)
  adding: res

In [0]:
!rm -r results
!mkdir results

In [0]:
def mapBboxToSingleCoord(locations):
  xs = list(map(lambda x: x[0],locations))
  ys = list(map(lambda x: x[1],locations))

  newX = (min(xs)+max(xs)) / 2
  newY = (min(ys)+max(ys)) / 2
  return (newX,newY)

def mapImagePixelToLatLon(city,id_,path,coord,lat,lon):
  targetFileGeo = path+id_+ ".tfw"
  Xm, Ym = convertPixelToMetreOffset(coord,targetFileGeo)
  newLatFromBase, newLonFromBase = transformLatLonByXMetres(lat,lon,Ym,Xm)
  return (newLatFromBase, newLonFromBase)

def getBaseLatLon(city):
  fileIds = findAllFileIds(city)
  for path,id_ in fileIds:
    with open(path+id_+".prj",'r') as f:
      s = f.read()
    lon = eval(re.findall(r"\[\"Central_Meridian\",.*?\]", s)[0])
    lat = eval(re.findall(r"\[\"Latitude_Of_Origin\",.*?\]", s)[0])
    if len(lon) > 0 and len(lat) > 0:
      return(lat[1],lon[1])

def convertPixelToMetreOffset(coord,targetFileGeo):
  pixelX, pixelY = coord
  with open(targetFileGeo, 'rb') as f:
    lines = f.readlines()
    lonOffsetFromBase = float(lines[-1].strip())
    latOffsetFromBase = float(lines[-2].strip())
    yPixelsToMetres = 1 / float(lines[-3].strip())
    xPixelsToMetres = 1 / float(lines[0].strip())

    metreOffsetOnY = pixelY / yPixelsToMetres
    metreOffsetOnX = pixelX / xPixelsToMetres

    return (metreOffsetOnX+latOffsetFromBase,metreOffsetOnY+lonOffsetFromBase)
  
def transformLatLonByXMetres(lat,lon,dn,de):
    R=6377563.396 #from their files
    dLat = dn/R
    dLon = de/(R*cos(pi*lat/180))
    latO = lat + dLat * 180/pi
    lonO = lon + dLon * 180/pi

    return (latO,lonO)


In [0]:
allImagesToText()